### Trabalho - Gustavo e Edilson

In [1]:
import os
import io
import numpy

NEWLINE = '\n'
SKIP_FILES = set(['cmds'])

def read_files(path):
    for root, dir_names, file_names in os.walk(path):
        for path in dir_names:
            read_files(os.path.join(root, path))
    for file_name in file_names:
        if file_name not in SKIP_FILES:
            file_path = os.path.join(root, file_name)
        if os.path.isfile(file_path):
            past_header, lines = False, []
            #f = open(file_path)
            f = io.open(file_path, mode="r", encoding="Latin-1")
            for line in f:
                if past_header:
                    lines.append(line)
                elif line == NEWLINE:
                    past_header = True
            f.close()
            yield file_path, NEWLINE.join(lines)#.decode('cp1252', 'ignore')

In [2]:
from pandas import DataFrame

def build_data_frame(path, classification):
    data_frame = DataFrame({'text': [], 'class': []})
    for file_name, text in read_files(path): # ERRO
        data_frame = data_frame.append(
            DataFrame({'text': [text], 'class': [classification]}, index=[file_name]))
    return data_frame

In [3]:
HAM = 0
SPAM = 1

SOURCES = [
    ('data/spam',         SPAM),
    ('data/spam_2',       SPAM),
    ('data/easy_ham',     HAM),
    ('data/hard_ham',     HAM),
    ]

data = DataFrame({'text': [], 'class': []})
for path, classification in SOURCES:
    data = data.append(build_data_frame(path, classification))

# data = data.reindex(numpy.random.permutation(data.index))

In [4]:
from sklearn.utils import shuffle
data = shuffle(data)

In [5]:
train = data.iloc[:4400]
test = data.iloc[4400:]

In [6]:
import numpy
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer()
counts = count_vectorizer.fit_transform(numpy.asarray(data['text']))

In [7]:
from sklearn.naive_bayes import MultinomialNB

classifier = MultinomialNB()
targets = numpy.asarray(data['class'])
classifier.fit(counts, targets)

MultinomialNB()

In [8]:
type(test['text'])

pandas.core.series.Series

In [9]:
examples = ['Free Viagra call today!', "I'm going to attend the Linux users group tomorrow."]
example_counts = count_vectorizer.transform(examples)
predictions = classifier.predict(example_counts)
predictions # [1, 0]

array([1., 0.])

In [10]:
example_counts = count_vectorizer.transform(test['text'])
predictions = classifier.predict(example_counts)
predictions

array([0., 1., 1., 0., 1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1.,
       1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 1., 1., 0., 1., 1.,
       1., 1., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 1., 1., 0., 0.,
       1., 0., 1., 1., 0., 1., 1., 0., 1., 1., 0., 1., 1., 0., 1., 0., 0.,
       0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0.,
       0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1.,
       0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 1., 1., 0., 0.,
       1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1.,
       1., 1., 1., 1., 0., 0., 0., 1., 0., 0., 1., 1., 0., 1., 1., 0., 1.,
       0., 1., 0., 0., 1., 0., 1., 0., 0., 1., 0., 1., 1., 0., 0., 1., 0.,
       0., 1., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       1., 0., 1., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1.,
       0., 1., 1., 1., 1.

In [11]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [12]:
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(test['class'], predictions)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(test['class'], predictions)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(test['class'], predictions)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(test['class'], predictions)
print('F1 score: %f' % f1)

Accuracy: 0.993333
Precision: 1.000000
Recall: 0.982759
F1 score: 0.991304
